In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchsummary import summary
torch.__version__

'1.9.1+cpu'

In [2]:
transform = transforms.Compose([
#     transforms.CenterCrop(224),

    transforms.RandomCrop(32,padding=4), # 数据增广
    transforms.RandomHorizontalFlip(),  # 数据增广
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
]) 

In [107]:
from PIL import Image
import torch
from torch.utils.data import Dataset


class MyDataSet(Dataset):
    """自定义数据集"""

    def __init__(self, images_path: list, images_class: list, transform=None):
        self.images_path = images_path
        self.images_class = images_class
        self.transform = transform

    def __len__(self):
        return len(self.images_path)

    def __getitem__(self, item):
        img = Image.open(self.images_path[item])
        print("kk", img)
        # RGB为彩色图片，L为灰度图片
        if img.mode != 'RGB':
            raise ValueError("image: {} isn't RGB mode.".format(self.images_path[item]))
        label = self.images_class[item]

        if self.transform is not None:
            img = self.transform(img)

        return img, label

#     @staticmethod
#     def collate_fn(batch):
#         # 官方实现的default_collate可以参考
#         # https://github.com/pytorch/pytorch/blob/67b7e751e6b5931a9f45274653f4f653a4e6cdf6/torch/utils/data/_utils/collate.py
#         images, labels = tuple(zip(*batch))

#         images = torch.stack(images, dim=0)
#         labels = torch.as_tensor(labels)
#         return images, labels


In [108]:
train_csv = pd.read_csv("./concat_train.csv")
len(train_csv)
train_csv.columns
x = int(len(train_csv)*0.8)
len(train_csv['image_id'][:x])
len(train_csv['image_id'][x:])

6650

In [109]:
train_images_path = []
train_images_label = []
test_images_path = []
test_images_label = []


for img_id in train_csv['image_id'][:x]:
    img_path = os.path.join('./train_image/',img_id)
    train_images_path.append(img_path)
    
for label in train_csv['label'][:x]:
    train_images_label.append(label)
    
for img_id in train_csv['image_id'][x:]:
    img_path = os.path.join('./train_image/',img_id)
    test_images_path.append(img_path)
    
for label in train_csv['label'][x:]:
    test_images_label.append(label)

In [110]:
train_images_path[:3]
test_images_label[:3]

[3, 3, 3]

In [111]:
train_dataset = MyDataSet(images_path=train_images_path,
                          images_class=train_images_label,
                          transform=transform)

test_dataset = MyDataSet(images_path=test_images_path,
                          images_class=test_images_label,
                          transform=transform)

In [112]:
batch_size = 256

In [113]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=0
                                           )
test_loader = torch.utils.data.DataLoader(test_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=0
                                           )

In [114]:
im,label = iter(train_loader).next()
print(len(im))

kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B9233320>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B940CE10>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B940CD30>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B940C710>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B940C860>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B940CE10>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B940CD30>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B940C710>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B940C860>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B940CE10>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B940CD30>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B9

kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B972FA58>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B972FA20>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B972F748>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B972F7F0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B972FA90>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B972FAC8>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B972FB38>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B972F908>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B972F7F0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B972FC88>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B972FC50>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B9

kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B9726080>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B9726470>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B9726048>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B9726390>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B9726320>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B9726470>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B97263C8>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B9726080>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B9726048>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B9726390>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B9726438>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B9

In [36]:
# img = Image.open('./train_image/9002905433618.jpg')
# img = img.load()

In [37]:
# print(img[0,0])

(25, 0, 5)


In [115]:
train_loader

In [116]:
class BasicBlock(nn.Module):
    """
    对于浅层网络，如ResNet-18/34等，用基本的Block
    基础模块没有压缩,所以expansion=1
    """
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock,self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(True),
            nn.Conv2d(out_channels,out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )
        # 如果输入输出维度不等，则使用1x1卷积层来改变维度
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels),
            )
    def forward(self, x):
        out = self.features(x)
#         print(out.shape)
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

In [117]:
# 测试
basic_block = BasicBlock(64, 128)
print(basic_block)
x = torch.randn(2, 64, 32, 32)
y = basic_block(x)
print(y.shape)

BasicBlock(
  (features): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (shortcut): Sequential(
    (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)
torch.Size([2, 128, 32, 32])


In [118]:
class Bottleneck(nn.Module):
    """
    对于深层网络，我们使用BottleNeck，论文中提出其拥有近似的计算复杂度，但能节省很多资源
    zip_channels: 压缩后的维数，最后输出的维数是 expansion * zip_channels
    针对ResNet50/101/152的网络结构,主要是因为第三层是第二层的4倍的关系所以expansion=4
    """
    expansion = 4
    
    def __init__(self, in_channels, zip_channels, stride=1):
        super(Bottleneck, self).__init__()
        out_channels = self.expansion * zip_channels
        self.features = nn.Sequential(
            nn.Conv2d(in_channels, zip_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(zip_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(zip_channels, zip_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(zip_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(zip_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
            
    def forward(self, x):
        out = self.features(x)
#         print(out.shape)
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

In [119]:
# 测试
bottleneck = Bottleneck(256, 128)
print(bottleneck)
x = torch.randn(2, 256, 32, 32)
y = bottleneck(x)
print(y.shape)

Bottleneck(
  (features): Sequential(
    (0): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (7): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (shortcut): Sequential(
    (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)
torch.Size([2, 512, 32, 32])


In [120]:
class ResNet(nn.Module):
    """
    不同的ResNet架构都是统一的一层特征提取、四层残差，不同点在于每层残差的深度。
    对于cifar10，feature map size的变化如下：
    (32, 32, 3) -> [Conv2d] -> (32, 32, 64) -> [Res1] -> (32, 32, 64) -> [Res2] 
 -> (16, 16, 128) -> [Res3] -> (8, 8, 256) ->[Res4] -> (4, 4, 512) -> [AvgPool] 
 -> (1, 1, 512) -> [Reshape] -> (512) -> [Linear] -> (10)
    """
    def __init__(self, block, num_blocks, num_classes=10, verbose = False):
        super(ResNet, self).__init__()
        self.verbose = verbose
        self.in_channels = 64
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        #使用_make_layer函数生成上表对应的conv2_x, conv3_x, conv4_x, conv5_x的结构
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        # cifar10经过上述结构后，到这里的feature map size是 4 x 4 x 512 x expansion
        # 所以这里用了 4 x 4 的平均池化
        self.avg_pool = nn.AvgPool2d(kernel_size=4)
        self.classifer = nn.Linear(512 * block.expansion, num_classes)
        
    def _make_layer(self, block, out_channels, num_blocks, stride):
        # 第一个block要进行降采样
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            # 如果是Bottleneck Block的话需要对每层输入的维度进行压缩，压缩后再增加维数
            # 所以每层的输入维数也要跟着变
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.features(x)
        if self.verbose:
            print('block 1 output: {}'.format(out.shape))
        out = self.layer1(out)        
        if self.verbose:
            print('block 2 output: {}'.format(out.shape))
        out = self.layer2(out)
        if self.verbose:
            print('block 3 output: {}'.format(out.shape))
        out = self.layer3(out)
        if self.verbose:
            print('block 4 output: {}'.format(out.shape))
        out = self.layer4(out)
        if self.verbose:
            print('block 5 output: {}'.format(out.shape))
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.classifer(out)
        return out

In [121]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' # 判断是否用GPU
device

'cpu'

In [122]:
def ResNet18(verbose=False):
    return ResNet(BasicBlock, [2,2,2,2],verbose=verbose)

def ResNet34(verbose=False):
    return ResNet(BasicBlock, [3,4,6,3],verbose=verbose)

def ResNet50(verbose=False):
    return ResNet(Bottleneck, [3,4,6,3],verbose=verbose)

def ResNet101(verbose=False):
    return ResNet(Bottleneck, [3,4,23,3],verbose=verbose)

def ResNet152(verbose=False):
    return ResNet(Bottleneck, [3,8,36,3],verbose=verbose)

net = ResNet18(True).to(device)

In [123]:
summary(net,(3,32,32))

block 1 output: torch.Size([2, 64, 32, 32])
block 2 output: torch.Size([2, 64, 32, 32])
block 3 output: torch.Size([2, 128, 16, 16])
block 4 output: torch.Size([2, 256, 8, 8])
block 5 output: torch.Size([2, 512, 4, 4])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          36,864
       BatchNorm2d-5           [-1, 64, 32, 32]             128
              ReLU-6           [-1, 64, 32, 32]               0
            Conv2d-7           [-1, 64, 32, 32]          36,864
       BatchNorm2d-8           [-1, 64, 32, 32]             128
        BasicBlock-9           [-1, 64, 32, 32]               0
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchN

In [124]:
net = ResNet34().to(device)
if device == 'cuda':
    net = nn.DataParallel(net)
    # 当计算图不会改变的时候（每次输入形状相同，模型不改变）的情况下可以提高性能，反之则降低性能
    torch.backends.cudnn.benchmark = True

In [125]:
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=1e-1, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5,verbose=True,patience = 5,min_lr = 0.000001) # 动态更新学习率
# scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[75, 150], gamma=0.5)

import time
epoch = 20

In [126]:
import os
if not os.path.exists('./model'):
    os.makedirs('./model')
else:
    print('文件已存在')
save_path = './model/ResNet34.pth'

文件已存在


In [ ]:
from utils import train
from utils import plot_history
Acc, Loss, Lr = train(net, train_loader, test_loader, epoch, optimizer, criterion, scheduler, save_path, verbose = True)

kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B470>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B160>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B2B0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B160>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B2B0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B160>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B470>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B128>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B358>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B940>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B160>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B9

kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B320>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B358>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B2B0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B470>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B128>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B320>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B358>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B0F0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B1D0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B320>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B208>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B9

kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B1D0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B320>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B128>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B978>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B1D0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B128>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B320>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B1D0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B128>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B320>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B992B1D0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B9

kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7264B38>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7264F98>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7264D68>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7264C18>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7264B38>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7264F98>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7264D68>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7264C18>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7264CC0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7264BE0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7264F98>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7

kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7264978>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7264A90>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7264BE0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7264A58>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7264978>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7264C50>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7264A90>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7264A58>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7264978>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7264C50>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7264A90>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7

...

kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202CC0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202BA8>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202B70>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202CF8>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B72028D0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202BA8>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202B70>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202CC0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B72028D0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202BA8>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202BE0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7

kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B72029B0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202C50>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202BE0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202E80>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202C50>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B72028D0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202DD8>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202E80>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202C50>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B72028D0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202DD8>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7

kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202F28>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202F60>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202E80>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202EB8>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202160>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202EB8>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202DD8>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202E80>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202F28>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202160>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7202EB8>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7

kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7242208>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7242198>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7242240>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7242128>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7242048>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7242240>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B72422B0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B72421D0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7242160>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7242240>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B72421D0>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7

kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7242160>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7242438>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7242278>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B72423C8>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7242240>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7242390>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7242400>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B72423C8>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7242240>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7242048>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7242358>
kk <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x1D5B7

In [ ]:
plot_history(epoch ,Acc, Loss, Lr)

### 测试

In [ ]:
correct = 0   # 定义预测正确的图片数，初始化为0
total = 0     # 总共参与测试的图片数，也初始化为0
# testloader = torch.utils.data.DataLoader(testset, batch_size=32,shuffle=True, num_workers=2)
torch.cuda.empty_cache()
net.eval()
with torch.no_grad():
    for data in testloader:  # 循环每一个batch
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        net.eval()  # 把模型转为test模式
        if hasattr(torch.cuda, 'empty_cache'):
            torch.cuda.empty_cache()
        outputs = net(images)  # 输入网络进行测试

        # outputs.data是一个4x10张量，将每一行的最大的那一列的值和序号各自组成一个一维张量返回，第一个是值的张量，第二个是序号的张量。
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)          # 更新测试图片的数量
        correct += (predicted == labels).sum() # 更新正确分类的图片的数量

    print('Accuracy of the network on the 10000 test images: %.2f %%' % (100 * correct / total))

In [ ]:
 # 定义2个存储每类中测试正确的个数的 列表，初始化为0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
# testloader = torch.utils.data.DataLoader(testset, batch_size=64,shuffle=True, num_workers=2)
net.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        if hasattr(torch.cuda, 'empty_cache'):
            torch.cuda.empty_cache()
        outputs = net(images)

        _, predicted = torch.max(outputs.data, 1)
    #4组(batch_size)数据中，输出于label相同的，标记为1，否则为0
        c = (predicted == labels).squeeze()
        for i in range(len(images)):      # 因为每个batch都有4张图片，所以还需要一个4的小循环
            label = labels[i]   # 对各个类的进行各自累加
            class_correct[label] += c[i]
            class_total[label] += 1
 
 
for i in range(10):
    print('Accuracy of %5s : %.2f %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()
images_ = images
#images_ = images_.view(images.shape[0], -1)
images_ = images_.to(device)
labels = labels.to(device)
val_output = net(images_)
_, val_preds = torch.max(val_output, 1)

fig = plt.figure(figsize=(25,4))

correct = torch.sum(val_preds == labels.data).item()

# val_preds = val_preds.cpu()
# labels = labels.cpu()

print("Accuracy Rate = {}%".format(correct/len(images) * 100))

fig = plt.figure(figsize=(25,25))
for idx in np.arange(64):    
    ax = fig.add_subplot(8, 8, idx+1, xticks=[], yticks=[])
    #fig.tight_layout()
#     plt.imshow(im_convert(images[idx]))
    imshow(images[idx])
    ax.set_title("{}, ({})".format(classes[val_preds[idx].item()], classes[labels[idx].item()]), 
                 color = ("green" if val_preds[idx].item()==labels[idx].item() else "red"))

### 保存模型

In [ ]:
torch.save(net,save_path[:-4]+'_'+str(epoch)+'.pth')
# torch.save(net, './model/ResNet34.pth')

### 预测

In [ ]:
import torch
from PIL import Image
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
 
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ResNet34()

model = torch.load(save_path)  # 加载模型
# model = model.to('cuda')
model.eval()  # 把模型转为test模式

# 读取要预测的图片
img = Image.open("./airplane.jpg").convert('RGB') # 读取图像

In [ ]:
img

In [ ]:
trans = transforms.Compose([transforms.Scale((32,32)),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=(0.5, 0.5, 0.5), 
                                                 std=(0.5, 0.5, 0.5)),
                           ])
 
img = trans(img)
img = img.to(device)
# 图片扩展多一维,因为输入到保存的模型中是4维的[batch_size,通道,长，宽]，而普通图片只有三维，[通道,长，宽]
img = img.unsqueeze(0)  
    # 扩展后，为[1，1，28，28]
output = model(img)
prob = F.softmax(output,dim=1) #prob是10个分类的概率
print("概率",prob)
value, predicted = torch.max(output.data, 1)
print("类别",predicted.item())
print(value)
pred_class = classes[predicted.item()]
print("分类",pred_class)